In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/ML Datasets/diabetes-dataset.csv")
dataset = dataset.sample(frac=1)

X = dataset.drop("Outcome", axis=1)
y = dataset["Outcome"]

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=10)

X_train = np.array(train_X)
X_test = np.array(test_X)
y_train = np.array(train_y)
y_test = np.array(test_y)

X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape(1, y_train.shape[0])
y_test = y_test.reshape(1, y_test.shape[0])

print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("y_test shape: " + str(y_test.shape))

In [ ]:
def structure(X, Y):
  input_size = X.shape[0]
  hidden_size = 8
  output_size = Y.shape[0]

  return input_size, hidden_size, output_size

In [ ]:
def initialize(input_size, hidden_size, output_size):
  W1 = np.random.randn(hidden_size, input_size) * 0.001
  b1 = np.zeros(shape=(hidden_size, 1))
  W2 = np.random.randn(output_size, hidden_size)
  b2 = np.zeros(shape=(output_size, 1))

  params = {"W1": W1,
            "W2": W2,
            "b1": b1,
            "b2": b2}

  return params

In [ ]:
def sigmoid(x):
  y = 1/(1+np.exp(-x))
  return y

In [ ]:
def forward_prop(X, params):
  W1 = params["W1"]
  b1 = params["b1"]
  W2 = params["W2"]
  b2 = params["b2"]

  Z1 = np.dot(W1, X) + b1
  A1 = np.tanh(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2) + b2

  vars = {"Z1": Z1,
          "A1": A1,
          "Z2": Z2,
          "A2": A2}
  #return output and dict containing internal variables
  return A2, vars

In [ ]:
def cost(A2, Y):
  m = Y.shape[1]
  cost = (-1/m) * np.sum(Y * np.log(A2) + (1-Y) * (np.log(1-A2)))

  cost = np.squeeze(cost)

  return cost

In [ ]:
def backprop(X, Y, vars, params):
  m = Y.shape[1]

  Z1 = vars["Z1"]
  A1 = vars["A1"]
  Z2 = vars["Z2"]
  A2 = vars["A2"]

  W1 = params["W1"]
  b1 = params["b1"]
  W2 = params["W2"]
  b2 = params["b2"]

  dZ2 = A2 - Y
  dW2 = (1/m) * np.dot(dZ2, A1.T)
  db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
  dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
  dW1 = (1/m) * np.dot(dZ1, X.T)
  db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

  grads = {"dZ2": dZ2,
           "dW2": dW2,
           "db2": db2,
           "dZ1": dZ1,
           "dW1": dW1,
           "db1": db1}
  return grads

In [ ]:
def update(params, grads, learning_rate):

  W1 = params["W1"]
  b1 = params["b1"]
  W2 = params["W2"]
  b2 = params["b2"]

  dZ2 = grads["dZ2"]
  dW2 = grads["dW2"]
  db2 = grads["db2"]
  dZ1 = grads["dZ1"]
  dW1 = grads["dW1"]
  db1 = grads["db1"]

  W1 = W1 - dW1 * learning_rate
  b1 = b1 - db1 * learning_rate
  W2 = W2 - dW2 * learning_rate
  b2 = b2 - db2 * learning_rate

  params = {"W1": W1,
            "W2": W2,
            "b1": b1,
            "b2": b2}

  return params

In [ ]:
def model(X, Y, iterations, learning_rate):

  params = initialize(structure(X,Y)[0], structure(X,Y)[1], structure(X,Y)[2])

  W1 = params["W1"]
  b1 = params["b1"]
  W2 = params["W2"]
  b2 = params["b2"]

  for i in range(iterations):
    A2, vars = forward_prop(X, params)
    cost_func = cost(A2, Y)
    grads = backprop(X, Y, vars, params)
    params = update(params, grads, learning_rate)
    if i % 500 == 0:
      print("Cost after iteration", i, ": ", cost_func)

  return params

In [ ]:
def predict(X, params):
  A2, vars = forward_prop(X, params)
  prediction = A2.round()

  return prediction

In [ ]:
def test(X1, y1, X2, y2, iterations, learning_rate):
  params = model(X1, y1, iterations, learning_rate)
  predictions = predict(X1, params)

  print("Training Accuracy: %d" % float((np.dot(y1, predictions.T) + np.dot(1 - y1, 1 - predictions.T)) / float(y1.size) * 100) + '%')

  predictions = predict(X2, params)

  print("Testing Accuracy: %d" % float((np.dot(y2, predictions.T) + np.dot(1 - y2, 1 - predictions.T)) / float(y2.size) * 100) + '%')


In [ ]:
test(X_train, y_train, X_test, y_test, 10000, 0.001)